<a href="https://colab.research.google.com/github/timescast/fantastic-rotary-phone-topic-modelling/blob/main/assignment_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Loading

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

docs = pd.read_csv('/content/drive/MyDrive/subset_data.csv')

In [ ]:
docs.head()


,Id,OwnerUserId,CreationDate,ClosedDate,Score,Title,Body,document,Tag
0,17016800,922360.0,2013-06-10T04:15:05Z,NaN,0,Handling the EditText send keyboard event for ...,<pre><code>import com.example.methanegaszonege...,Handling the EditText send keyboard event for ...,android events android-edittext send
1,7685280,315306.0,2011-10-07T09:20:41Z,NaN,7,EditText: how to enable/disable input?,<p>I have a 7x6 grid of EditText views. I want...,EditText: how to enable/disable input? <p>I ha...,android
2,24178500,1397271.0,2014-06-12T07:13:00Z,NaN,1,Mobile web - Displaying a fixed div below a re...,<p>I want to have a relative div at the top of...,Mobile web - Displaying a fixed div below a re...,jquery html css iphone mobile
3,38820760,4397012.0,2016-08-08T03:10:28Z,NaN,0,How to create tabbed view in HTML?,<p>I'm trying to create a tabbed view in HTML ...,How to create tabbed view in HTML? <p>I'm tryi...,html google-sites
4,3674120,443107.0,2010-09-09T05:53:46Z,NaN,0,Problems decrypting HTTP Live Stream,<p>I have a single key encrypted HTTP Live Str...,Problems decrypting HTTP Live Stream <p>I have...,http stream openssl live encryption


In [ ]:

titles = docs['Title'].tolist()
bodies = docs['Body'].tolist()
tags = docs['Tag'].tolist()
TITLE_WEIGHT = 1
BODY_WEIGHT = 2

In [ ]:
weighted_documents = []
for title, body in zip(titles, bodies):

    weighted_title_part = (' ' + title) * TITLE_WEIGHT
    weighted_body_part = (' ' + body) * BODY_WEIGHT

    weighted_doc = f"{weighted_title_part.strip()} {weighted_body_part.strip()}"
    weighted_documents.append(weighted_doc)



# Dense Vectors

In [ ]:
from sentence_transformers import SentenceTransformer

embeding_model = SentenceTransformer("all-MiniLM-L6-v2")

In [ ]:
embeddings = embeding_model.encode(weighted_documents,show_progress_bar=True)

In [ ]:
embeddings

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


 Saving Embeddings

In [ ]:
import numpy as np

np.save('/content/drive/MyDrive/embeddings.npy', embeddings)

Load Embedding

In [ ]:
import numpy as np
saved_embeddings = np.load('/content/drive/MyDrive/embeddings.npy')

In [ ]:
import numpy as np
saved_embeddings = np.load('/content/drive/MyDrive/embed_weighted_desc.npy')

In [ ]:
!pip install -q bertopic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.0/153.0 kB 5.2 MB/s eta 0:00:00


# Intialize Flow

In [ ]:
from umap import UMAP
from hdbscan import HDBSCAN
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sentence_transformers import SentenceTransformer
from bertopic import BERTopic
from bertopic.vectorizers import ClassTfidfTransformer
from bertopic.representation import KeyBERTInspired


embeddings = saved_embeddings

embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine')


hdbscan_model = HDBSCAN(min_cluster_size=15, metric='euclidean', cluster_selection_method='eom', prediction_data=True)


vectorizer_model = CountVectorizer(stop_words="english",min_df=10)

representation_model = KeyBERTInspired()

ctfidf_model = ClassTfidfTransformer(
    bm25_weighting=True,
    reduce_frequent_words=True
)


# All steps together
topic_model = BERTopic(
  embedding_model=embedding_model,
  umap_model=umap_model,
  hdbscan_model=hdbscan_model,
  vectorizer_model=vectorizer_model,
  ctfidf_model=ctfidf_model,
  representation_model=representation_model,
  calculate_probabilities=True,
  verbose=True
)

In [ ]:
topics, probs = topic_model.fit_transform(weighted_documents, embeddings=saved_embeddings)

2025-11-10 08:03:39,826 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-11-10 08:03:54,498 - BERTopic - Dimensionality - Completed ✓
2025-11-10 08:03:54,500 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-11-10 08:03:54,890 - BERTopic - Cluster - Completed ✓
2025-11-10 08:03:54,896 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-11-10 08:04:06,733 - BERTopic - Representation - Completed ✓


In [ ]:

topic_model.save("/content/drive/MyDrive/models/model00", serialization="safetensors")

In [ ]:

loaded_model = BERTopic.load("/content/drive/MyDrive/models")

2025-11-07 07:14:02,152 - BERTopic - WARNING: You are loading a BERTopic model without explicitly defining an embedding model. If you want to also load in an embedding model, make sure to use `BERTopic.load(my_model, embedding_model=my_embedding_model)`.


In [ ]:
topic_model.get_document_info(weighted_documents)

In [ ]:
freq = topic_model.get_topic_info()
freq.head(5)

In [ ]:
topic_model.get_topic(0)

[('php', np.float32(0.22129023)),
 ('framework', np.float32(0.1828571)),
 ('config', np.float32(0.17137566)),
 ('logged', np.float32(0.16780773)),
 ('module', np.float32(0.1571293)),
 ('login', np.float32(0.14042929)),
 ('adding', np.float32(0.13228804)),
 ('added', np.float32(0.12857017)),
 ('ids', np.float32(0.11735826)),
 ('execute', np.float32(0.10768563))]

# Diagnostic and Visualization

In [ ]:
topic_model.visualize_topics()

In [ ]:
topic_model.visualize_barchart()

In [ ]:
topic_model.visualize_heatmap()

# Handling Outliers

In [ ]:
from umap import UMAP
from hdbscan import HDBSCAN
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer

from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired
from bertopic.vectorizers import ClassTfidfTransformer


embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
#changing n_neighbors to 10 and n_components 2
umap_model = UMAP(n_neighbors=10, n_components=5, min_dist=0.0, metric='cosine')

hdbscan_model = HDBSCAN(min_cluster_size=10,min_samples=2, metric='euclidean', cluster_selection_method='eom', prediction_data=True)


vectorizer_model = CountVectorizer(stop_words="english",min_df=10)

ctfidf_model = ClassTfidfTransformer(
    bm25_weighting=True,
    reduce_frequent_words=True
)

representation_model = KeyBERTInspired()

topic_model = BERTopic(
    embedding_model=embedding_model,
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    vectorizer_model=vectorizer_model,
    ctfidf_model=ctfidf_model,
    representation_model=representation_model,
    calculate_probabilities=True,
    verbose=True
)

In [ ]:
reduced_embeddings = UMAP(n_neighbors=10, n_components=2,
                          min_dist=0.0, metric='cosine').fit_transform(saved_embeddings)

In [ ]:
topics, probs = topic_model.fit_transform(weighted_documents,saved_embeddings)

2025-11-10 09:06:01,270 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-11-10 09:06:12,796 - BERTopic - Dimensionality - Completed ✓
2025-11-10 09:06:12,797 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-11-10 09:06:25,178 - BERTopic - Cluster - Completed ✓
2025-11-10 09:06:25,187 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-11-10 09:06:42,996 - BERTopic - Representation - Completed ✓


In [ ]:
topic_model.visualize_documents(weighted_documents, reduced_embeddings=reduced_embeddings, hide_annotations=True)

In [ ]:
new_topics = topic_model.reduce_outliers(weighted_documents, topics, probabilities=probs, strategy="probabilities")

In [ ]:
topic_model.update_topics(weighted_documents, topics=new_topics)

2025-11-10 09:10:39,151 - BERTopic - WARNING: Using a custom list of topic assignments may lead to errors if topic reduction techniques are used afterwards. Make sure that manually assigning topics is the last step in the pipeline.Note that topic embeddings will also be created through weightedc-TF-IDF embeddings instead of centroid embeddings.


In [ ]:
topic_model.visualize_documents(weighted_documents, reduced_embeddings=reduced_embeddings, hide_annotations=True)

In [ ]:
freq = topic_model.get_topic_info()
freq.head(5)

,Topic,Count,Name,Representation,Representative_Docs
0,0,169,0_rb_lib_ruby_rails,"[rb, lib, ruby, rails, gems, 00, end, actionpa...",[ActiveRecord::NoDatabaseError: local user wit...
1,1,162,1_binding_asp_grid_runat,"[binding, asp, grid, runat, listbox, textblock...",[WPF Data binding alternate datacontext inside...
2,2,154,2_view_self_tableview_frame,"[view, self, tableview, frame, cell, uiviewcon...",[Add multiple custom UIViewControllers to UIPa...
3,3,150,3_div_width_background_css,"[div, width, background, css, height, margin, ...",[Extending Background Color Beyond Wrapper <p>...
4,4,167,4_database_dbo_sql_go,"[database, dbo, sql, go, table, sqlite, db, co...",[How do I execute a .SQL file from Excel VBA c...


In [ ]:
topic_model.visualize_barchart()

In [ ]:
topic_model.visualize_topics()

In [ ]:

hierarchical_topics = topic_model.hierarchical_topics(weighted_documents)

topic_model.visualize_hierarchy(hierarchical_topics=hierarchical_topics)

100%|██████████| 206/206 [00:03<00:00, 63.59it/s]


In [ ]:
import pandas as pd



df_topics = pd.DataFrame({
    'Document': weighted_documents,
    'Topic': topics
})

topic_names = topic_model.get_topic_info().set_index('Topic')['Name'].to_dict()
df_topics['Topic_Name'] = df_topics['Topic'].map(topic_names)

print("DataFrame created successfully.")

DataFrame created successfully.


In [ ]:

target_topic_id = 5

topic_documents = df_topics[df_topics['Topic'] == target_topic_id]

print(f"-----Topic {target_topic_id}: {topic_documents['Topic_Name'].iloc[0]}")
print(f"Keywords: {topic_model.get_topic(target_topic_id)}")
print("\n--- Sample Documents ---")


for i, row in topic_documents.sample(5).iterrows():

    print(f"[{i}] {row['Document'][:300]}...")

# Random Case

In [ ]:
from umap import UMAP
from hdbscan import HDBSCAN
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer

from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired
from bertopic.vectorizers import ClassTfidfTransformer


embedding_model = SentenceTransformer("all-MiniLM-L6-v2")


vectorizer_model = CountVectorizer(stop_words="english",min_df=10)

ctfidf_model = ClassTfidfTransformer(
    bm25_weighting=True,
    reduce_frequent_words=True
)

representation_model = KeyBERTInspired()



In [ ]:

umap_model = UMAP(n_neighbors=10, n_components=2, min_dist=0.0, metric='cosine')

hdbscan_model = HDBSCAN(min_cluster_size=10,min_samples=2, metric='euclidean', cluster_selection_method='eom', prediction_data=True)
topic_model = BERTopic(
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    verbose=True,
    calculate_probabilities=True
)

In [ ]:
topic_model.fit(weighted_documents, embeddings=saved_embeddings)

2025-11-11 09:45:34,212 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-11-11 09:45:44,880 - BERTopic - Dimensionality - Completed ✓
2025-11-11 09:45:44,882 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-11-11 09:46:06,144 - BERTopic - Cluster - Completed ✓
2025-11-11 09:46:06,162 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-11-11 09:46:10,571 - BERTopic - Representation - Completed ✓


In [ ]:
topics_found = topic_model.get_topic_freq().shape[0] - 1

In [ ]:
topics_found

247

In [ ]:
type(freq)

In [ ]:
topic_embeddings = topic_model.topic_embeddings_

In [ ]:
topic_embeddings.shape

(260, 384)

In [ ]:
freq.to_csv("/content/drive/MyDrive/topic_info.csv", index=False)

In [ ]:
freq = pd.read_csv("/content/drive/MyDrive/topic_info.csv")

Random data set load

In [ ]:
import pandas as pd

df = pd.read_csv("/content/sample_data/Questions.csv", skiprows=range(1, 8000), nrows=50)
print(df)


In [ ]:
df.to_csv("/content/drive/MyDrive/subset_data_random.csv", index=False)


In [ ]:
import pandas as pd
r_docs = pd.read_csv('/content/drive/MyDrive/subset_data_random.csv')

In [ ]:
r_docs.head()

,Id,OwnerUserId,CreationDate,ClosedDate,Score,Title,Body
0,533180,NaN,2009-02-10T16:52:32Z,NaN,1,Joining fact tables in an MDX query,<p>I am building and Anaysis Services project ...
1,533200,14395.0,2009-02-10T16:56:00Z,NaN,6,allow .NET 2.0 runtime to run executables from...,"<p>Guys, this can't be for real</p>\n\n<p>I'm ..."
2,533260,9178.0,2009-02-10T17:10:17Z,NaN,7,Ninject : Resolving an object by type _and_ re...,<p>I am looking for a way to do something like...
3,533300,5287.0,2009-02-10T17:18:53Z,NaN,15,What does 'self' refer to in a @classmethod?,"<p>I thought I was starting to get a grip on ""..."
4,533310,27521.0,2009-02-10T17:21:47Z,NaN,3,Viewing Rails model page works on first view b...,<p>This is using Rails 2.2.2</p>\n\n<p>I have ...


In [ ]:
titles = r_docs['Title'].tolist()
bodies = r_docs['Body'].tolist()

TITLE_WEIGHT = 1
BODY_WEIGHT = 2

In [ ]:
r_weighted_documents = []
for title, body in zip(titles, bodies):

    weighted_title_part = (' ' + title) * TITLE_WEIGHT
    weighted_body_part = (' ' + body) * BODY_WEIGHT

    weighted_doc = f"{weighted_title_part.strip()} {weighted_body_part.strip()}"
    r_weighted_documents.append(weighted_doc)

In [ ]:
randwdoc = [r_weighted_documents[0]]

In [ ]:
rembeddings = embeding_model.encode(r_weighted_documents,show_progress_bar=True)

In [ ]:
import numpy as np

np.save('/content/drive/MyDrive/embeddings_random.npy', rembeddings)

In [ ]:
import numpy as np
r_saved_embeddings = np.load('/content/drive/MyDrive/embeddings_random.npy')

load end

In [ ]:
topic_embeddings = topic_model.topic_embeddings_
topic_embeddings = topic_model.umap_model.transform(topic_embeddings)
new_doc_embedding_reduced = topic_model.umap_model.transform(r_saved_embeddings)

In [ ]:
sim_matrix = cosine_similarity(new_doc_embedding_reduced, topic_embeddings)

In [ ]:
predictions = np.argmax(sim_matrix, axis=1)
confidences = np.max(sim_matrix, axis=1)

In [ ]:
predictions

array([174, 158, 109, 175,  35,  10,   5,  52,  50, 188,  33, 171, 104,
        15, 106, 204, 153,  17, 193, 166,  37,  64, 167,  14,  84,  17,
        19, 156, 103,  91,  65, 146, 184,  88, 121, 142, 196,  84, 159,
       148, 233, 119,   2, 121,  65, 199, 202,  77, 159, 204])

In [ ]:
confidences

array([0.99999994, 0.99999976, 0.99999857, 1.        , 0.99999994,
       0.99997437, 0.99999684, 0.999992  , 0.9999999 , 0.999997  ,
       0.9999991 , 0.9999992 , 0.9999988 , 0.99999887, 0.9999984 ,
       0.9999997 , 0.9999992 , 0.9999983 , 0.9999977 , 0.9999999 ,
       0.99999976, 0.9999998 , 0.99999774, 1.        , 1.0000001 ,
       0.99999875, 0.99999976, 0.9999988 , 1.        , 0.9999965 ,
       0.99999964, 0.9999997 , 0.9999807 , 0.99999624, 0.99999815,
       0.9999994 , 0.9999998 , 1.        , 0.9999999 , 0.99999994,
       1.        , 0.99999815, 0.999999  , 0.99999905, 0.9999997 ,
       0.9999951 , 0.99999577, 0.99999195, 0.9999996 , 0.9999916 ],
      dtype=float32)

In [ ]:
df_transform = pd.DataFrame({
    'New_Document': r_weighted_documents,
    'Predicted_Topic_ID': predictions,
    'Confidence (Max Cos Sim)': confidences.round(4)
})

In [ ]:
topic_names = {t: topic_model.get_topic_info(t)['Name'].iloc[0] if t != -1 else 'Outliers'
               for t in np.unique(predictions)}
# topic_names = {t: topic_model.get_topic_info(t)['Name'].iloc[0]
#                for t in np.unique(predictions)}

df_transform['Predicted_Topic_Name'] = df_transform['Predicted_Topic_ID'].map(topic_names)

In [ ]:
df_transform

,New_Document,Predicted_Topic_ID,Confidence (Max Cos Sim),Predicted_Topic_Name
0,Joining fact tables in an MDX query <p>I am bu...,173,1.0,173_validresult_session_lua_iis
1,allow .NET 2.0 runtime to run executables from...,157,1.0,157_udp_readimu_res_whileres
2,Ninject : Resolving an object by type _and_ re...,108,1.0,108_root_20_5551m_18g
3,What does 'self' refer to in a @classmethod? <...,174,1.0,174_mod_3233_2753_3120
4,Viewing Rails model page works on first view b...,34,1.0,34_django_def_model_kwargs
5,Dynamic dispatch and binding <p>Are dynamic di...,9,1.0,9_htaccess_rewriterule_rewritecond_redirect
6,Tools for programming tutorials? <p>I engage i...,4,1.0,4_echo_gt_ltoption_ltphp
7,Creating a class to use for populating drop-do...,51,1.0,51_form_validation_submit_ltinput
8,Is there a way to avoid errors when releasing ...,49,1.0,49_loadedgroupindex_finalcheck_nsstring_become...
9,Next and Previous to navigate text fields <p>I...,187,1.0,187_pagerect_cimg_image_bmp


In [ ]:
new_topic, new_prob = topic_model.transform(r_weighted_documents,embeddings=r_saved_embeddings)


2025-11-11 09:55:22,317 - BERTopic - Dimensionality - Reducing dimensionality of input embeddings.
2025-11-11 09:55:22,354 - BERTopic - Dimensionality - Completed ✓
2025-11-11 09:55:22,355 - BERTopic - Clustering - Approximating new points with `hdbscan_model`
2025-11-11 09:55:22,365 - BERTopic - Probabilities - Start calculation of probabilities with HDBSCAN
2025-11-11 09:55:22,508 - BERTopic - Probabilities - Completed ✓
2025-11-11 09:55:22,510 - BERTopic - Cluster - Completed ✓


In [ ]:
new_topic

[np.int64(-1),
 np.int64(-1),
 np.int64(93),
 np.int64(56),
 np.int64(208),
 np.int64(-1),
 np.int64(-1),
 np.int64(-1),
 np.int64(49),
 np.int64(14),
 np.int64(32),
 np.int64(228),
 np.int64(-1),
 np.int64(-1),
 np.int64(105),
 np.int64(-1),
 np.int64(60),
 np.int64(-1),
 np.int64(-1),
 np.int64(-1),
 np.int64(-1),
 np.int64(70),
 np.int64(144),
 np.int64(-1),
 np.int64(-1),
 np.int64(34),
 np.int64(18),
 np.int64(-1),
 np.int64(228),
 np.int64(90),
 np.int64(-1),
 np.int64(35),
 np.int64(32),
 np.int64(226),
 np.int64(49),
 np.int64(-1),
 np.int64(216),
 np.int64(-1),
 np.int64(158),
 np.int64(188),
 np.int64(76),
 np.int64(-1),
 np.int64(-1),
 np.int64(-1),
 np.int64(106),
 np.int64(198),
 np.int64(57),
 np.int64(-1),
 np.int64(158),
 np.int64(203)]

In [ ]:
fitted = []
for n in range(len(new_topic)):
  if new_topic[n] != -1:
    fitted.append(new_topic[n])

In [ ]:
fitted[0]

np.int64(10)

In [ ]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,1684,-1_at_to_the_is,"[at, to, the, is, in, and, of, codepre, this, it]",[Android stand alone service <p>I just started...
1,0,134,0_char_struct_int_mov,"[char, struct, int, mov, ltlt, stack, cout, in...",[how to add String by value and not reference ...
2,1,117,1_isystemout19770_0628_210556610_210556615,"[isystemout19770, 0628, 210556610, 210556615, ...",[How to access c# visual studio project from d...
3,2,108,2_androidlayoutheightwrapcontent_androidlayout...,"[androidlayoutheightwrapcontent, androidlayout...",[Items of ListView in ListActivity remain uncl...
4,3,105,3_width_ltdiv_div_ltdivgt,"[width, ltdiv, div, ltdivgt, background, heigh...",[position relative content calculate height <p...
...,...,...,...,...,...
229,228,10,228_xul_net_language_vms,"[xul, net, language, vms, applications, thunde...",[Performance Calling C from C# <p>Edit: To nar...
230,229,10,229_i18nstrings_array_gt_perl,"[i18nstrings, array, gt, perl, suffix, advance...",[Issues with Powershell variables/expressions ...
231,230,10,230_ltnavigationcasegt_xhtml_ignoredupresponse...,"[ltnavigationcasegt, xhtml, ignoredupresponse,...",[JSF questions and advices <p>I'm doing a JSF2...
232,231,10,231_domain_logginginfo_domainemail_predomainad...,"[domain, logginginfo, domainemail, predomainad...",[Best way to submit a form each time a section...


In [ ]:
freq = pd.read_csv("/content/drive/MyDrive/topic_info.csv")

# Cosine Similarity

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity